<a href="https://colab.research.google.com/github/zzzguy/-/blob/main/Python_%EC%A3%BC%EC%8B%9D_%EC%8A%A4%ED%81%AC%EB%A6%AC%EB%84%88_(%EA%B8%B0%EB%8A%A5_%EA%B0%95%ED%99%94_%EB%B2%84%EC%A0%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이 코드는 pykrx 라이브러리를 사용하여 한국 주식 정보를 다룹니다.
# --------------------------------------------------------------------
# 실행 전, 터미널(CMD)에 아래 명령어를 먼저 실행해주세요.
# pip install pandas pykrx
# --------------------------------------------------------------------
import sys
from datetime import datetime, timedelta
import pandas as pd

# --- 1. 필수 라이브러리 확인 ---
try:
    from pykrx import stock
except ImportError:
    print("\n" + "="*60)
    print("⚠️ 필수 라이브러리가 설치되지 않았습니다!")
    print("   아래 명령어를 복사하여 터미널(CMD)에 붙여넣고 실행해주세요.")
    print("\n   pip install pandas pykrx\n")
    print("="*60)
    sys.exit()

def format_market_cap(cap):
    """시가총액을 조, 억 단위로 변환하는 함수"""
    if not isinstance(cap, (int, float)) or cap == 0:
        return "N/A"
    trillion = int(cap / 10**12)
    billion = int((cap % 10**12) / 10**8)
    if trillion > 0:
        return f"{trillion}조 {billion}억"
    return f"{billion}억"

def run_screener():
    """사용자 설정에 따라 주식 종목을 스크리닝하는 메인 함수"""
    print("==============================================")
    print("      KRX 주식 스크리너 (Advanced)")
    print("==============================================")

    # --- 2. 사용자 설정 입력 ---
    # 1. 기준일자 선택
    while True:
        date_input = input("🔍 기준일자를 입력하세요 (예: 20230510) | 미입력 시 최신 영업일: ")
        if not date_input:
            date_input = datetime.now().strftime('%Y%m%d')
            target_date = stock.get_nearest_business_day_in_a_week(date_input)
            print(f"   -> 최신 영업일인 {target_date}로 설정됩니다.")
            break
        try:
            # 입력된 날짜가 유효한지 확인
            datetime.strptime(date_input, '%Y%m%d')
            target_date = date_input
            break
        except ValueError:
            print("   -> ⚠️ 잘못된 형식입니다. YYYYMMDD 형식으로 다시 입력해주세요.")

    # 2. 시장 선택
    while True:
        market_choice = input("🏢 시장을 선택하세요 (1: 전체, 2: 코스피, 3: 코스닥): ")
        if market_choice in ['1', '2', '3']:
            break
        print("   -> ⚠️ 1, 2, 3 중에서 선택해주세요.")

    # 3. 52주 신고가 돌파 여부 체크
    while True:
        high_52w_check = input("📈 52주 신고가 돌파 종목만 보시겠습니까? (y/n): ").lower()
        if high_52w_check in ['y', 'n']:
            break
        print("   -> ⚠️ y 또는 n으로 입력해주세요.")

    print("\n[ 검색 시작 ]")
    print("----------------------------------------------")

    # --- 3. 데이터 가져오기 및 처리 ---
    try:
        # 시장 선택에 따른 티커 목록 가져오기
        markets = []
        if market_choice == '1':
            markets.extend(["KOSPI", "KOSDAQ"])
        elif market_choice == '2':
            markets.append("KOSPI")
        else:
            markets.append("KOSDAQ")

        all_tickers = []
        for market in markets:
            print(f"⏳ {market} 시장의 종목 정보를 가져옵니다...")
            tickers = stock.get_market_ticker_list(target_date, market=market)
            all_tickers.extend(tickers)

        print(f"✅ 총 {len(all_tickers)}개 종목을 대상으로 검색을 시작합니다.")

        # 기준일의 시가총액, 가격 정보 일괄 조회
        df_cap = stock.get_market_cap(target_date)
        df_ohlcv = stock.get_market_ohlcv(target_date)

        # 두 데이터를 합치고 필요한 컬럼만 선택
        df_base = pd.merge(df_ohlcv, df_cap[['시가총액']], left_index=True, right_index=True, how='inner')
        df_base = df_base[df_base.index.isin(all_tickers)]

        screened_list = []

        # 52주 신고가 체크 로직
        if high_52w_check == 'y':
            print("⏳ 52주 신고가 여부를 확인합니다. (시간이 소요될 수 있습니다)")
            start_52w = (datetime.strptime(target_date, '%Y%m%d') - timedelta(days=365)).strftime('%Y%m%d')

            for i, ticker in enumerate(df_base.index):
                # 진행 상황 표시
                progress = f"   ({i + 1}/{len(df_base)}) {ticker} 확인 중..."
                sys.stdout.write(f"\r{progress}")
                sys.stdout.flush()

                try:
                    df_52w = stock.get_market_ohlcv(start_52w, target_date, ticker)
                    if df_52w.empty: continue

                    # 기준일의 고가가 52주 최고가(고가 기준)와 같거나 높으면 신고가로 판단
                    if df_52w.iloc[-1]['고가'] >= df_52w['고가'].max():
                        screened_list.append(ticker)
                except:
                    continue # 개별 종목 조회 실패 시 건너뛰기

            # 필터링된 종목들만 남김
            result_df = df_base[df_base.index.isin(screened_list)].copy()
            sys.stdout.write("\r" + " " * 50 + "\r") # 진행 상황 표시 줄 지우기
        else:
            result_df = df_base.copy()

        # 종목명 추가
        result_df['종목명'] = result_df.index.map(lambda x: stock.get_market_ticker_name(x))

    except Exception as e:
        print(f"❌ 데이터를 처리하는 중 오류가 발생했습니다: {e}")
        return

    # --- 4. 최종 결과 출력 ---
    print("\n==============================================")
    print("🎉 최종 검색 결과 🎉")
    print("==============================================")

    if result_df.empty:
        print("아쉽지만, 조건에 맞는 종목을 찾지 못했습니다. 😥")
    else:
        # 결과 컬럼 정리 및 포매팅
        result_df['시가총액'] = result_df['시가총액'].apply(format_market_cap)
        result_df.rename(columns={'종가': '당일종가', '등락률': '전일대비 증감율(%)'}, inplace=True)

        final_df = result_df[['종목명', '시가총액', '당일종가', '전일대비 증감율(%)']]

        print(f"총 {len(final_df)}개의 종목을 찾았습니다!")
        print(final_df.to_string(index=False))


if __name__ == "__main__":
    run_screener()

      KRX 주식 스크리너 (Advanced)
🔍 기준일자를 입력하세요 (예: 20230510) | 미입력 시 최신 영업일: 
   -> 최신 영업일인 20250804로 설정됩니다.
🏢 시장을 선택하세요 (1: 전체, 2: 코스피, 3: 코스닥): 2
📈 52주 신고가 돌파 종목만 보시겠습니까? (y/n): Y

[ 검색 시작 ]
----------------------------------------------
⏳ KOSPI 시장의 종목 정보를 가져옵니다...
✅ 총 962개 종목을 대상으로 검색을 시작합니다.
⏳ 52주 신고가 여부를 확인합니다. (시간이 소요될 수 있습니다)
   (838/962) 006200 확인 중...